In [2]:
%pip install -U langchain-community langgraph langchain-anthropic tavily-python langgraph-checkpoint-sqlite

   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.5 MB ? eta -:--:--
   ---------------- ----------------------- 1.0/2.5 MB 2.6 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 4.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 24.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1734355274, 'localtime': '2024-12-16 05:21'}, 'current': {'last_updated_epoch': 1734354900, 'last_updated': '2024-12-16 05:15', 'temp_c': 6.7, 'temp_f': 44.1, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 5.8, 'wind_kph': 9.4, 'wind_degree': 156, 'wind_dir': 'SSE', 'pressure_mb': 1021.0, 'pressure_in': 30.14, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 89, 'cloud': 75, 'feelslike_c': 4.8, 'feelslike_f': 40.7, 'windchill_c': 8.3, 'windchill_f': 46.9, 'heatindex_c': 9.5, 'heatindex_f': 49.1, 'dewpoint_c': 9.3, 'dewpoint_f': 48.7, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.0, 'gust_mph': 9.6, 'gust_kph': 15.5}}"}, {'url': 'https://weathershogun.com/weather/usa/

In [5]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini",base_url="https://aihubmix.com/v1")

In [6]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

APIStatusError: Error code: 405